# Training & Testing Dataset Preparation

### Metadata

In [ ]:
import os
import numpy

import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# read in train data
TRAINING_DATA_PATH = '../2025_A2/train'
TRAIN_LABEL_PATH = TRAINING_DATA_PATH + '/train_metadata.csv'
IMAGE_DIMENTION = (64, 64)

### Function Definitions



In [ ]:
def get_labels(labels_path):
    # load labels as dict 
    labels = {}
    mxm_labels = 0
    with open(labels_path, 'r') as f:
        counter:int = 0
        for line in f:
            # skip the first line
            if counter == 0:
                counter += 1
                continue
            line = line.strip().split(',')
            # line [1] is the image name, line [2] is the label
            labels[line[1]] = int(line[2])
            if int(line[2]) > mxm_labels:
                mxm_labels = int(line[2])
    print(f"Max label: {mxm_labels}")
    return labels

def load_and_preprocess_images(image_folder, labels_dict, target_size=IMAGE_DIMENTION):
    images = []
    labels = []
    
    for filename in os.listdir(image_folder):
        if filename.endswith(".jpg"): 
            img_path = os.path.join(image_folder, filename)
            
            # Read image with TensorFlow
            img = tf.io.read_file(img_path)  # Read the image file
            img = tf.image.decode_jpeg(img, channels=3)  # Decode the JPEG image (for JPG files)

            # Resize the image to the target size (100x100)
            img_resized = tf.image.resize(img, target_size)
            
            # Normalize image to [0, 1]
            img_normalized = img_resized / 255.0
            images.append(img_normalized)
            
            # Get the label from the dictionary
            label = labels_dict.get(filename)
            labels.append(label)
    
    # Convert lists to numpy arrays
    images = numpy.array(images)
    labels = numpy.array(labels)
    
    return images, labels

### Prepare Data

In [ ]:
overall_images, overall_labels = load_and_preprocess_images(TRAINING_DATA_PATH, get_labels(TRAIN_LABEL_PATH))

train_images, test_images = tf.keras.utils.split_dataset(
    overall_images, left_size=0.9)

train_labels, test_labels = tf.keras.utils.split_dataset(
    overall_labels, left_size=0.9)


Max label: 42


2025-05-17 00:30:12.888603: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-05-17 00:30:12.888622: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 36.00 GB
2025-05-17 00:30:12.888625: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 13.50 GB
I0000 00:00:1747405812.888636 23205255 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1747405812.888652 23205255 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Model Definition & Model Training

### Definition

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(32, (9, 9), activation='relu', 
                        input_shape=(IMAGE_DIMENTION[0], IMAGE_DIMENTION[1], 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (6, 6), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
model.summary()

/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 56, 56, 32)     │         7,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 23, 23, 64)     │        73,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 11, 11, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 9, 9, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 43)             │         2,795 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 186,923 (730.17 KB)

 Trainable params: 186,923 (730.17 KB)

 Non-trainable params: 0 (0.00 B)

### Train

In [ ]:
# 1. Log everything to TensorBoard
tb = tf.keras.callbacks.TensorBoard(log_dir="logs", histogram_freq=1, write_images=True)

# tarin the model
model.fit(numpy.array(list(train_images)),
          numpy.array(list(train_labels)),
          epochs=10, batch_size=64, callbacks=[tb],
          validation_data=(numpy.array(list(train_images)), numpy.array(list(train_labels))))

2025-05-17 00:30:29.481095: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
2025-05-17 00:30:29.620486: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Epoch 1/10


2025-05-17 00:30:29.926256: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
/opt/anaconda3/envs/MPS-Torch/lib/python3.12/site-packages/keras/src/backend/tensorflow/nn.py:708: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(
2025-05-17 00:30:30.211598: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 44ms/step - accuracy: 0.0672 - loss: 3.6308 - val_accuracy: 0.3011 - val_loss: 2.8239
Epoch 2/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.4188 - loss: 2.4219 - val_accuracy: 0.6104 - val_loss: 1.5194
Epoch 3/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.6505 - loss: 1.3444 - val_accuracy: 0.7552 - val_loss: 0.8849
Epoch 4/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.7527 - loss: 1.0236 - val_accuracy: 0.8245 - val_loss: 0.6953
Epoch 5/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.8490 - loss: 0.6139 - val_accuracy: 0.8822 - val_loss: 0.4416
Epoch 6/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.8832 - loss: 0.4789 - val_accuracy: 0.9186 - val_loss: 0.3003
Epoch 7/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 43ms/step - accuracy: 0.8933 - loss: 0.4238 - val_accuracy: 0.9093 - val_loss: 0.3716
Epoch 8/10
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.9052 - loss: 0.4377 - val_accuracy: 0.9354 - val_loss: 0.

# Evaluation

In [ ]:
validation_loss, validation_accuracy = model.evaluate(numpy.array(list(test_images)), numpy.array(list(test_labels)), batch_size=32)

print(f"Validation Loss: {validation_loss}")
print(f"Validation Accuracy: {validation_accuracy}")

18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8834 - loss: 0.6680
Validation Loss: 0.6155427098274231
Validation Accuracy: 0.8888888955116272
